In [1]:
import numpy as np
import pandas as pd
import re
import calendar

import seaborn as sns
import matplotlib.style as style


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

style.use('fivethirtyeight')

In [2]:
df = pd.read_csv('outputs/job_postings_data_scientist_sf.csv')
df.head()

,job_title,salary,company,location,is_remote,job_rating,job_summary,post_date,extract_date,job_url
0,Data Scientist,NaN,Thermo Fisher - America,"San Francisco, CA",NaN,3.5,Organize and data mine clinical data to assess...,3 days ago,2020-11-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Data Scientist,NaN,"Alto Neuroscience, Inc","San Francisco Bay Area, CA",Remote,NaN,This person will work closely with a growing i...,30+ days ago,2020-11-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,Data Scientist,"$250,000 - $375,000 a year",Blue Owl,"San Francisco, CA",Temporarily remote,NaN,"We’re using rich customer insights, advanced t...",30+ days ago,2020-11-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,Data Scientist,$80 - $120 an hour,Braintrust,"San Francisco, CA",Remote,4.5,Analyze data for trends and patterns and inter...,2 days ago,2020-11-27,https://www.indeed.com/rc/clk?jk=e8f687ad91faa...
4,"Data Scientist, Analyst - Invest",NaN,SoFi,"San Francisco, CA",NaN,3.1,Build data structures and ETL pipelines to tra...,1 day ago,2020-11-27,https://www.indeed.com/rc/clk?jk=2d2b4e08ab4ce...
